# Load libraries

In [ ]:
pip install pyheif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 3.0 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import imgaug.augmenters as iaa
import numpy as np
from PIL import Image
import pyheif

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_dir = "/content/drive/My Drive/SMU MITB/5th Term/CS604: Deep Learning for Visual Recognition/Group Project/Prof Samples"
resize_dir = "/content/drive/My Drive/SMU MITB/5th Term/CS604: Deep Learning for Visual Recognition/Group Project/Prof Samples/1. Resized"
aug_dir = "/content/drive/My Drive/SMU MITB/5th Term/CS604: Deep Learning for Visual Recognition/Group Project/Prof Samples/2. Augmented"

# Helper functions

In [ ]:
def read_jpg(file_path):
    image = cv2.imread(file_path)
    return image

In [ ]:
def read_heic(file_path):

    # Read the HEIC file
    heif_file = pyheif.read(file_path)

    # Convert HEIC to PIL Image
    image = Image.frombytes(heif_file.mode, heif_file.size, heif_file.data, "raw", heif_file.mode, heif_file.stride)

    # Convert PIL Image to OpenCV format
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    return image

In [ ]:
def read_webp(file_path):

    # Use Pillow to open the WEBP file
    image_pil = Image.open(file_path).convert('RGB')

    # Convert the PIL Image to a NumPy array
    image_np = np.array(image_pil)

    # Convert RGB to BGR format for OpenCV
    image_cv = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    return image_cv

# Downsize images

In [ ]:
def resize_image(image, target_size = None, resize_factor = None):
    if target_size is not None:
        resized_image = cv2.resize(image, target_size, interpolation = cv2.INTER_AREA)
    elif resize_factor is not None:
        new_width = int(image.shape[1] * resize_factor)
        new_height = int(image.shape[0] * resize_factor)
        resized_image = cv2.resize(image, (new_width, new_height), interpolation = cv2.INTER_AREA)
    else:
        resized_image = image
    return resized_image

In [ ]:
target_size = (640, 640)

for img_file in os.listdir(input_dir):
    img_path = os.path.join(input_dir, img_file)
    if os.path.isfile(img_path):

        # Determine the file extension and select the appropriate function
        if img_file.lower().endswith('.jpg') or img_file.lower().endswith('.jpeg'):
            image = read_jpg(img_path)
        elif img_file.lower().endswith('.heic'):
            image = read_heic(img_path)
        elif img_file.lower().endswith('.webp'):
            image = read_webp(img_path)
        else:
            continue

        image_resized = resize_image(image, target_size = target_size)

        # Save the resized image
        resized_filename = "{}_resized.jpg".format(os.path.splitext(img_file)[0])
        resized_file_path = os.path.join(resize_dir, resized_filename)
        cv2.imwrite(resized_file_path, image_resized)

# Data Augmentation

In [ ]:
# Augmentation sequence
seq = iaa.Sequential([
    iaa.Fliplr(0.5),                                              # Horizontal flips
    iaa.Crop(percent = (0, 0.1)),                                 # Random crops
    iaa.Sometimes(0.5, iaa.GaussianBlur(sigma = (0, 0.5))),       # Blur 50% of all images with random sigma between 0 and 0.5
    iaa.LinearContrast((0.75, 1.5)),                              # Strengthen / Weaken the contrast in each image
    iaa.Affine(rotate = (-25, 25)),                               # Rotate between -25 and +25 degrees
    iaa.Affine(                                                   # Apply affine transformations: scale and translate
        scale = {"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent = {"x": (-0.2, 0.2), "y": (-0.2, 0.2)}
    )
], random_order = True)                                           # Apply augmenters in random order

In [ ]:
# Number of augmented versions per original image
num_augmented_versions = 11

for img_file in os.listdir(resize_dir):
    img_path = os.path.join(resize_dir, img_file)
    if os.path.isfile(img_path) and img_file.endswith('_resized.jpg'):
        image = cv2.imread(img_path)

        for i in range(num_augmented_versions):
            image_aug = seq(image = image)

            # Save the augmented image
            output_filename = "{}_aug_{}.jpg".format(os.path.splitext(img_file)[0], i+1)
            output_path = os.path.join(aug_dir, output_filename)
            cv2.imwrite(output_path, image_aug)